In [1]:
import os
os.chdir('../')
os.getcwd()

'e:\\github_clone\\Mixture-of-LLM-Expert'

In [2]:
import os
import re
import json
from MoE import SparseMoELanguageModel, kaiming_init_weights
from Utils import model_params
from Tokenizer import WordLevelTokenizer, extract_and_save_text, LetterLevelTokenizer
import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device
max_iters = 100000
learning_rate = 3e-4
block_size = 8
batch_size = 8
eval_iters = 500
n_emb = 384
n_layers = 6
n_head = 6

In [5]:
csv_file_path = 'data\\newspaper-text-summarization-cnn-dailymail\\cnn_dailymail\\test.csv'  # Replace with your CSV file path
output_directory = 'data\\'  # Replace with your desired output directory

extract_and_save_text(csv_file_path, output_directory)

In [6]:
with open("data\\summaries.txt", "rb") as txt:
    texts = txt.read()

In [7]:
chars = sorted(list(set(texts.decode())))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;=?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz £¥°´·½ÂáãäåçèéëíîïñóöûüćčēłŠšž​‎–—‘’“”…€
129


In [8]:
tokenizer = LetterLevelTokenizer()

In [9]:
temp = "Im the President of the America."

tokens = tokenizer.tokenize(temp)
print("Tokens:", tokens)  # Output: Tokens: [index values representing each word]

original_text = tokenizer.detokenize(tokens)
print("Detokenized text:", original_text)

Tokens: [34, 12, 94, 19, 7, 4, 94, 41, 17, 4, 18, 8, 3, 4, 13, 19, 94, 14, 5, 94, 19, 7, 4, 94, 26, 12, 4, 17, 8, 2, 0, 75]
Detokenized text: Im the President of the America.


In [10]:
data = tokenizer.tokenize(texts.decode())

In [11]:
encoded_data = torch.tensor(data, dtype=torch.long)
print(encoded_data.shape, encoded_data.dtype)

torch.Size([3545403]) torch.int64


In [12]:
n = int(0.9*len(encoded_data))
train_data = encoded_data[:n]
val_data = encoded_data[n:]

print(f"Training tokens : {len(train_data)}  --- Validation tokens : {len(val_data)}")

Training tokens : 3190862  --- Validation tokens : 354541


In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([30]) the target: 23
when input is tensor([30, 23]) the target: 15
when input is tensor([30, 23, 15]) the target: 4
when input is tensor([30, 23, 15,  4]) the target: 17
when input is tensor([30, 23, 15,  4, 17]) the target: 19
when input is tensor([30, 23, 15,  4, 17, 19]) the target: 18
when input is tensor([30, 23, 15,  4, 17, 19, 18]) the target: 94
when input is tensor([30, 23, 15,  4, 17, 19, 18, 94]) the target: 16


In [14]:
torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(encoded_data) - block_size, (batch_size,))
    x = torch.stack([encoded_data[i:i+block_size] for i in ix])
    y = torch.stack([encoded_data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("Inputs: ")
print(xb)
print("Targets: ")
print(yb)

Inputs: 
tensor([[ 6, 17, 24, 68, 73, 94,  2, 14],
        [ 4,  4, 13, 94, 19, 14, 94, 21],
        [13,  3, 18,  7,  8, 15, 94,  2],
        [11,  8, 13,  6, 94, 58, 74, 24],
        [ 7, 17,  4,  4, 94,  0, 11, 11],
        [24, 94,  8, 13, 94,  2, 14, 13],
        [ 3, 94, 14, 19,  7,  4, 17, 94],
        [ 3,  4, 94,  8, 13, 94, 48,  8]])
Targets: 
tensor([[17, 24, 68, 73, 94,  2, 14, 20],
        [ 4, 13, 94, 19, 14, 94, 21,  8],
        [ 3, 18,  7,  8, 15, 94,  2,  7],
        [ 8, 13,  6, 94, 58, 74, 24,  4],
        [17,  4,  4, 94,  0, 11, 11,  4],
        [94,  8, 13, 94,  2, 14, 13, 13],
        [94, 14, 19,  7,  4, 17, 94,  0],
        [ 4, 94,  8, 13, 94, 48,  8, 13]])


In [15]:
model = SparseMoELanguageModel(vocab_size)
model.apply(kaiming_init_weights)
model.to(device)

SparseMoELanguageModel(
  (token_embedding_table): Embedding(129, 368)
  (position_embedding_table): Embedding(32, 368)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=368, out_features=46, bias=False)
            (query): Linear(in_features=368, out_features=46, bias=False)
            (value): Linear(in_features=368, out_features=46, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=368, out_features=368, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (smoe): SparseMoE(
        (router): NoisyTopkRouter(
          (topkroute_linear): Linear(in_features=368, out_features=8, bias=True)
          (noise_linear): Linear(in_features=368, out_features=8, bias=True)
        )
        (experts): ModuleList(
          (0-7): 8 x Expert(
            (net): Sequential(
            

In [16]:
model_params(model)

Total parameters: 73,957,953
Trainable parameters: 73,957,953
Non-trainable parameters: 0


In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X, Y = X.to(device), Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"steps: {iter} train loss: {losses['train']} val loss: {losses['val']}")
        
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

steps: 0 train loss: 5.77391242980957 val loss: 5.754223346710205
steps: 500 train loss: 2.616345167160034 val loss: 2.6125969886779785
steps: 1000 train loss: 2.4855411052703857 val loss: 2.469041347503662
steps: 1500 train loss: 2.386364698410034 val loss: 2.3921549320220947
steps: 2000 train loss: 2.356753349304199 val loss: 2.3495171070098877
steps: 2500 train loss: 2.3321874141693115 val loss: 2.3349690437316895
steps: 3000 train loss: 2.28495192527771 val loss: 2.2820966243743896
steps: 3500 train loss: 2.2662851810455322 val loss: 2.2679355144500732
steps: 4000 train loss: 2.226921319961548 val loss: 2.2384283542633057
steps: 4500 train loss: 2.229139804840088 val loss: 2.22273850440979
steps: 5000 train loss: 2.1903066635131836 val loss: 2.209552049636841
steps: 5500 train loss: 2.1668858528137207 val loss: 2.1823906898498535
steps: 6000 train loss: 2.157639265060425 val loss: 2.17037296295166
steps: 6500 train loss: 2.132796049118042 val loss: 2.1502015590667725
steps: 7000 tr

# 726 mins

In [19]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_word = model.generate(context, max_new_tokens=50)
seq = " Batman is the "
for i in generated_word.tolist():
    seq = seq + ""+tokenizer.detokenize(i)

In [20]:
seq

' Batman is the anged thee beimpey ftowistowegeepoweendered boreqrt'

In [21]:
torch.save(model.state_dict(), "artifacts\\moe_letter.pth")